In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
from pandas import *
import pandas as pd
date = datetime.today().strftime('%Y_%m_%d')


In [0]:
tables = {}
table_list = spark.sql("SHOW TABLES IN workspace.`tegge-insurance-data`").toPandas()
for table in table_list['tableName']:
    if table.endswith('_bronze'):
        key = table.replace('_bronze', '_silver')
    else:
        key = table
    tables[key] = f"workspace.`tegge-insurance-data`.{table}"

In [0]:
dfs = {}
for df_name, table in tables.items():
    dfs[df_name] = spark.read.format("delta").table(table).where(f"load_timestamp LIKE '%{date}%'")


In [0]:
dfs["pharmacy_claims_silver"] = dfs["pharmacy_claims_silver"].withColumnRenamed("rx_claim_id", "pharmacy_claim_id")
dfs["gl_transactions_silver"] = dfs["gl_transactions_silver"].withColumnRenamed("gl_txn_id", "gl_transaction_id")

In [0]:
df_schemas = {}
for df_name, df in dfs.items():
    try:
        df_schemas[df_name] = df.schema.fieldNames()
    except Exception as e:
        print(f"Could not read schema for {df_name}: {e}")

In [0]:
for table_name, df in dfs.items():
    if "employer_id" in df_schemas[table_name]:
        df = df.fillna({"employer_id": 9999})
    dfs[table_name] = df

In [0]:
pk_dict = spark.read.format("delta").table("workspace.`tegge-insurance-data-silver`.table_pk_fk")

pk_dict = pk_dict.select("Table_Name", "Primary_Key").toPandas()

pk_dict = dict(zip(pk_dict["Table_Name"] , pk_dict["Primary_Key"]))


In [0]:
fk_dict_df = spark.read.format("delta").table("workspace.`tegge-insurance-data-silver`.table_pk_fk")

fk_dict_df = fk_dict_df.select("Table_Name", "Foreign_Key_1", "Foreign_Key_2", "Foreign_Key_3", "Foreign_Key_4", "Foreign_Key_5").toPandas()

fk_dict = {}
for idx, row in fk_dict_df.iterrows():
    # Collect all non-null, non-empty foreign keys for this table
    fks = [row[f"Foreign_Key_{i}"] for i in range(1, 6) if pd.notnull(row[f"Foreign_Key_{i}"]) and str(row[f"Foreign_Key_{i}"]).strip() != ""]
    fk_dict[row["Table_Name"]] = fks

In [0]:
# Process all tables in dfs
for table_name, df in dfs.items():
    # Step 1: Identify rows with null primary key
    pk_col = pk_dict.get(table_name)
    if pk_col:
        null_pk_df = df.filter(col(pk_col).isNull())
        globals()[f"{table_name}_null_pks"] = null_pk_df
        df = df.filter(col(pk_col).isNotNull())
    
    # Step 2: Identify rows with null foreign key(s)
    fk_cols = fk_dict.get(table_name)
    if fk_cols:
        for fk_col in fk_cols:
            null_fk_df = df.filter(col(fk_col).isNull())
            globals()[f"{table_name}_null_fks_{fk_col}"] = null_fk_df
            #df = df.filter(col(fk_col).isNotNull())
    
    # Update dfs with cleaned DataFrame
    dfs[table_name] = df

In [0]:
for table_name in dfs.keys():
    null_pk_df = globals().get(f"{table_name}_null_pks")
    if null_pk_df is not None and null_pk_df.count() > 0:
        null_pk_df.write.format("delta").mode("overwrite").saveAsTable(f"workspace.`tegge-insurance-data-anomalies`.null_pk_{table_name}_{date}")
    null_fk_df = globals().get(f"{table_name}_null_fks")
    if null_fk_df is not None and null_fk_df.count() > 0:
        null_fk_df.write.format("delta").mode("overwrite").saveAsTable(f"workspace.`tegge-insurance-data-anomalies`.null_fk_{table_name}_{date}")

In [0]:
# Read unique identifier mapping table
unique_id_df = spark.read.format("delta").table("workspace.`tegge-insurance-data-silver`.table_unique_identifier")
unique_id_pd = unique_id_df.toPandas()

# Build mapping: table_name -> list of unique identifier columns
unique_id_map = {}
for idx, row in unique_id_pd.iterrows():
    table = row['Table_Name']
    # Collect all non-null, non-empty unique identifier columns
    unique_cols = [row[f"unique_identifier_{i}"] for i in range(1, 6) if pd.notnull(row.get(f"unique_identifier_{i}")) and str(row.get(f"unique_identifier_{i}")).strip() != ""]
    unique_id_map[table] = unique_cols

# Identify duplicates using PK + unique identifiers, save to Delta tables, and remove from dfs
duplicate_pkuid_tables = {}
for table_name, df in dfs.items():
    pk_col = pk_dict.get(table_name)
    uid_cols = unique_id_map.get(table_name, [])
    group_cols = []
    if pk_col and pk_col in df.columns:
        group_cols.append(pk_col)
    for col_name in uid_cols:
        if col_name in df.columns:
            group_cols.append(col_name)
    if group_cols:
        dup_df = df.groupBy(*group_cols).count().filter(col("count") > 1)
        if dup_df.count() > 0:
            # Join back to original DataFrame to get all duplicate records
            dup_keys = dup_df.select(*group_cols)
            dup_records = df.join(dup_keys, on=group_cols, how="inner")
            dup_records.write.format("delta").mode("overwrite").saveAsTable(f"workspace.`tegge-insurance-data-anomalies`.dup_{table_name}_{date}")
            print(f"Duplicates saved for {table_name} based on PK+UID: {group_cols}")
            # Remove duplicates from DataFrame
            df = df.dropDuplicates(group_cols)
            dfs[table_name] = df

In [0]:
# # Check for duplicate records and stop execution if any are found
# stop_execution = False
# stop_tables = []
# for table_name in dfs.keys():
#     null_pk_df = globals().get(f"{table_name}_null_pks")
#     if null_pk_df is not None and null_pk_df.count() > 0:
#         stop_execution = True
#         stop_tables.append(f"{table_name} (null PK)")

# if stop_execution:
#     raise RuntimeError(f"Null PK/FK detected in tables: {', '.join(stop_tables)}. Notebook execution stopped.")

In [0]:
# import smtplib
# from email.message import EmailMessage

# def send_alert(subject, body, to_email):
#     msg = EmailMessage()
#     msg.set_content(body)
#     msg['Subject'] = subject
#     msg['From'] = 'your_email@example.com'
#     msg['To'] = to_email
#     with smtplib.SMTP('smtp.yourprovider.com') as s:
#         s.send_message(msg)

# alert_needed = False
# alert_tables = []
# for table_name in dfs.keys():
#     if globals().get(f"{table_name}_null_pks") is not None and globals()[f"{table_name}_null_pks"].count() > 0:
#         alert_needed = True
#         alert_tables.append(f"{table_name} (null PK)")
#     if globals().get(f"{table_name}_null_fks") is not None and globals()[f"{table_name}_null_fks"].count() > 0:
#         alert_needed = True
#         alert_tables.append(f"{table_name} (null FK)")

# if alert_needed:
#     send_alert(
#         subject="Null PK/FK Detected in Silver Tables",
#         body=f"Tables with null PK/FK: {', '.join(alert_tables)}",
#         to_email="analyst@example.com"
#     )

In [0]:
# # Check for null PK/FK and stop execution if any are found
# stop_execution = False
# stop_tables = []
# for table_name in dfs.keys():
#     null_pk_df = globals().get(f"{table_name}_null_pks")
#     if null_pk_df is not None and null_pk_df.count() > 0:
#         stop_execution = True
#         stop_tables.append(f"{table_name} (null PK)")

# if stop_execution:
#     raise RuntimeError(f"Null PK/FK detected in tables: {', '.join(stop_tables)}. Notebook execution stopped.")

In [0]:
# PySpark does not support changing column nullability directly in-place on serverless compute.
# The best workaround is to use SQL ALTER TABLE statements to set columns as NOT NULL.
for table_name, df in dfs.items():
    pk_col = pk_dict.get(table_name)
    fk_cols = fk_dict.get(table_name)
    # Set PK column to NOT NULL
    if pk_col and pk_col in df.columns:
        spark.sql(f"ALTER TABLE workspace.`tegge-insurance-data`.{table_name} ALTER COLUMN {pk_col} SET NOT NULL")
    # Set FK columns to NOT NULL
    if fk_cols:
        for fk_col in fk_cols:
            if fk_col in df.columns:
                spark.sql(f"ALTER TABLE workspace.`tegge-insurance-data`.{table_name} ALTER COLUMN {fk_col} SET NOT NULL")